In [63]:
import baseline
import importlib
importlib.reload(baseline)
from baseline import *
import numpy as np 
import pandas as pd

In [64]:
#Data loading and reduce memory usage by changing dtypes
files = ['train.csv', 'test.csv', 'shops.csv', 'items.csv', 'item_categories.csv']

data = [loader(file_name) for file_name in files]

In [65]:
full_data, train = prepare_full_data(*data)

c:\Users\User\OneDrive\Рабочий стол\Trainee\baseline.py:77: FutureWarning: The behavior of pd.concat with len(keys) != len(objs) is deprecated. In a future version this will raise instead of truncating to the smaller of the two sequences
  full_data = pd.concat([full_data, test], keys=columns, ignore_index=True, sort=False)


In [66]:
reduce_mem_usage(full_data)
full_data.info()

Mem. usage decreased to 105.42 Mb (82.8% reduction)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11054182 entries, 0 to 11054181
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   date_block_num     int8   
 1   shop_id            int8   
 2   item_cnt_month     float16
 3   item_id            int16  
 4   city_id            int8   
 5   item_category_id   int8   
 6   main_category_id   int8   
 7   minor_category_id  int8   
dtypes: float16(1), int16(1), int8(6)
memory usage: 105.4 MB


In [67]:
#Check if our train data is valid
column_types = {'date_block_num': 'int8', 'shop_id': 'int8', 'city_id': 'int8', 'item_id': 'int16', 'item_cnt_month': 'float16',
'item_category_id': 'int8', 'main_category_id': 'int8', 'minor_category_id': 'int8'}
values_ranges = {'date_block_num': (0, 34), 'shop_id': (0, 59), 'item_id': (0, 22169), 'item_cnt_month': (0, 669), 'city_id':(0,30),
                'item_category_id': (0,83), 'main_category_id': (0,11), 'minor_category_id': (0, 66)}
Validator(column_types = column_types, value_ranges = values_ranges, check_missing = True, check_duplicates=True).fit_transform(full_data)

'Data is valid'

Feature exctraction

In [68]:
feature_extractor = FeatureExtractor(full_data, train)
full_featured_data = feature_extractor.process()

c:\Users\User\OneDrive\Рабочий стол\Trainee\baseline.py:527: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  self.full_data['months_from_first_sale'] = self.full_data['date_block_num'] - self.full_data.groupby(['shop_id', 'item_id'])['date_block_num'].transform('min')


In [69]:
reduce_mem_usage(full_featured_data)
full_featured_data.info()

Mem. usage decreased to 685.24 Mb (48.8% reduction)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11054182 entries, 0 to 11054181
Data columns (total 35 columns):
 #   Column                                Dtype  
---  ------                                -----  
 0   date_block_num                        int8   
 1   shop_id                               int8   
 2   item_cnt_month                        float16
 3   item_id                               int16  
 4   city_id                               int8   
 5   item_category_id                      int8   
 6   main_category_id                      int8   
 7   minor_category_id                     int8   
 8   shop_history                          int8   
 9   item_history                          float16
 10  minor_category_history                int8   
 11  first_sales_date_block                float16
 12  avg_item_cnt_per_cat_lag_1            float16
 13  avg_item_cnt_per_shop_lag_1           float16
 14  avg_item_cnt

In [70]:
full_featured_data.to_csv('full_data.csv', index = False)

Train/Test split

In [ ]:
#Validation
tss = TimeSeriesSplit(n_splits=3)

X_test = full_data[full_data.date_block_num == 34].drop('item_cnt_month', axis = 1)

X = full_data[full_data.date_block_num != 34].drop('item_cnt_month', axis = 1)
y = full_data[full_data.date_block_num != 34]['item_cnt_month']
tss = TimeSeriesSplit(n_splits=3)

for train_idxs, val_idxs in tss.split(X):

    X_train, X_val = X.iloc[train_idxs], X.iloc[val_idxs]
    y_train, y_val = y.iloc[train_idxs], y.iloc[val_idxs]
    

In [ ]:
#Training
X_train = full_data[~full_data.date_block_num.isin([33,34])]
y_train = X_train['item_cnt_month']
del X_train['item_cnt_month']

X_val = full_data[full_data['date_block_num']==33]
y_val = X_val['item_cnt_month']
del X_val['item_cnt_month']

X_test = full_data[full_data['date_block_num']==34].drop(columns='item_cnt_month')
X_test = X_test.reset_index()
del X_test['index']